# [Lab2] 모델 훈련 with SageMaker Training

이 노트북에서는 SageMaker Training Job을 사용하여 XGBoost 모델을 훈련합니다.

## 주요 내용
- SageMaker Built-in 알고리즘을 사용한 모델 훈련
- 다양한 하이퍼파라미터 설정으로 모델 비교
- MLflow를 통한 실험 추적 및 모델 등록
- Script Mode를 사용한 커스텀 훈련
- 자동 로깅 기능 활용

## 1. 환경 설정 및 변수 로드

In [ ]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")
print(f"   - S3 버킷: {bucket}")
print(f"   - 훈련 데이터 경로: {train_path}")
print(f"   - 검증 데이터 경로: {validation_path}")
print(f"   - 실험 이름: {experiment_name}")

In [ ]:
# 필수 라이브러리 임포트
import sagemaker
import boto3
import mlflow
import os
from time import gmtime, strftime
from sagemaker.xgboost.estimator import XGBoost
from IPython.display import Javascript

# AWS 세션 초기화
boto_session = boto3.Session()
sess = sagemaker.Session()

print("✅ 라이브러리 임포트 및 세션 초기화 완료")

## 2. 훈련 데이터 준비

전처리된 데이터를 SageMaker Training Job에서 사용할 수 있도록 설정합니다.

In [ ]:
# XGBoost 컨테이너 이미지 URI 가져오기
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.7-1'
)

print(f"✅ XGBoost 컨테이너 이미지: {container}")

In [ ]:
# 훈련 입력 데이터 설정
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=train_path, 
    content_type='csv'
)
s3_input_validation = sagemaker.inputs.TrainingInput(
    s3_data=validation_path, 
    content_type='csv'
)

training_inputs = {
    'train': s3_input_train, 
    'validation': s3_input_validation
}

print("✅ 훈련 입력 데이터 설정 완료")
print(f"   - 훈련 데이터: {train_path}")
print(f"   - 검증 데이터: {validation_path}")

## 3. MLflow 실험 설정

모델 훈련 과정을 추적하기 위한 MLflow 설정을 수행합니다.

In [ ]:
# 프로젝트 및 MLflow 설정
from sagemaker_studio import Project
from time import gmtime, strftime
import mlflow

# 프로젝트 설정 (MLflow 제외)
project = Project()
role = project.iam_role
domain_id = project.domain_id
project_id = project.id



print(f"   - MLflow 서버: {mlflow_name}")
mlflow.set_tracking_uri(mlflow_arn)

# 실행 이름 생성
run_suffix = strftime('%d-%H-%M-%S', gmtime())
run_name = f"data-preprocessing-{run_suffix}"

# MLflow 실행 시작
run_id = mlflow.start_run(
    run_name=run_name, 
    description="SageMaker Processing을 사용한 데이터 전처리"
).info.run_id

print(f"✅ MLflow 실행 시작: {run_name}")
print(f"   - 실행 ID: {run_id}")

## 4. Built-in 알고리즘을 사용한 모델 훈련

SageMaker의 Built-in XGBoost 알고리즘을 사용하여 두 가지 다른 하이퍼파라미터 설정으로 모델을 훈련합니다.

In [ ]:
# 첫 번째 모델 설정 (보수적인 하이퍼파라미터)
xgb1 = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role, 
    instance_count=1, 
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/output/',
    base_job_name='conservative-xgb-training',
    sagemaker_session=sess
)

# 보수적인 하이퍼파라미터 설정
xgb1.set_hyperparameters(
    max_depth=3,
    eta=0.5,
    gamma=4,
    eval_metric="auc",
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    objective='binary:logistic',
    num_round=50
)

print("✅ 첫 번째 모델 (보수적 설정) 구성 완료")
print(f"   - 학습률(eta): 0.5")
print(f"   - 최대 깊이: 3")
print(f"   - 라운드 수: 50")

In [ ]:
# 두 번째 모델 설정 (적극적인 하이퍼파라미터)
xgb2 = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role, 
    instance_count=1, 
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/output/',
    base_job_name='aggressive-xgb-training',
    sagemaker_session=sess
)

# 적극적인 하이퍼파라미터 설정
xgb2.set_hyperparameters(
    max_depth=3,
    eta=0.1,
    gamma=2,
    eval_metric="auc",
    min_child_weight=3,
    subsample=0.4,
    verbosity=0,
    objective='binary:logistic',
    num_round=100
)

print("✅ 두 번째 모델 (적극적 설정) 구성 완료")
print(f"   - 학습률(eta): 0.1")
print(f"   - 최대 깊이: 3")
print(f"   - 라운드 수: 100")

## 5. MLflow 실험 시작 및 모델 훈련

MLflow 실험을 시작하고 두 모델을 훈련합니다.

In [ ]:
# 실행 이름 생성
mlflow.end_run()

# 실행 이름 생성
run_suffix = strftime('%d-%H-%M-%S', gmtime())
run_name = f"model-training-{run_suffix}"

# MLflow 실행 시작
run_id = mlflow.start_run(
    run_name=run_name, 
    description="SageMaker Built-in XGBoost를 사용한 모델 훈련"
).info.run_id

print(f"✅ MLflow 실행 시작: {run_name}")
print(f"   - 실행 ID: {run_id}")

In [ ]:
# 첫 번째 모델 훈련 시작 (비동기)
print("🚀 첫 번째 모델 훈련 시작 (백그라운드)...")
xgb1.fit(training_inputs, wait=False, logs=False)

print(f"✅ 첫 번째 모델 훈련 작업 시작: {xgb1.latest_training_job.name}")

In [ ]:
# 두 번째 모델 훈련 시작 (동기)
print("🚀 두 번째 모델 훈련 시작 (대기)...")
xgb2.fit(training_inputs, wait=True, logs=False)

print(f"✅ 두 번째 모델 훈련 완료: {xgb2.latest_training_job.name}")

## ⏳ 훈련 진행 중...

모델 훈련이 진행 중입니다. 완료될 때까지 잠시 기다려주세요.

- 첫 번째 모델: 보수적 설정으로 빠른 훈련
- 두 번째 모델: 적극적 설정으로 정밀한 훈련

## 6. MLflow 수동 로깅

현재 실행 중인 MLflow 실험에 훈련된 모델의 정보를 로깅합니다.

In [ ]:
def load_model(model_data_s3_uri):
    """S3에서 훈련된 XGBoost 모델을 로드하는 함수"""
    import xgboost as xgb
    import tarfile
    import pickle as pkl

    model_file = "./xgboost-model.tar.gz"
    bucket, key = model_data_s3_uri.replace("s3://", "").split("/", 1)
    boto3.client("s3").download_file(bucket, key, model_file)
    
    with tarfile.open(model_file, "r:gz") as t:
        t.extractall(path=".")
    
    # 모델 로드
    model = xgb.Booster()
    model.load_model("xgboost-model")

    return model

print("✅ 모델 로딩 함수 정의 완료")

In [ ]:
# MLflow 실행 상태 확인 및 정리
try:
    # 현재 활성화된 실행이 있는지 확인
    active_run = mlflow.active_run()
    if active_run:
        print(f"⚠️ 활성화된 MLflow 실행 발견: {active_run.info.run_id}")
        if active_run.info.run_id != run_id:
            print("   다른 실행이 활성화되어 있습니다. 종료합니다.")
            mlflow.end_run()
        else:
            print("   현재 실행과 동일합니다. 계속 진행합니다.")
    else:
        print("✅ 활성화된 MLflow 실행이 없습니다.")
except Exception as e:
    print(f"MLflow 상태 확인 중 오류: {e}")
    # 안전하게 모든 실행 종료
    try:
        mlflow.end_run()
    except:
        pass

In [ ]:
# 현재 활성화된 MLflow 실행에 두 모델의 정보 로깅
# 이미 활성화된 실행이 있으므로 새로운 start_run 없이 직접 로깅

# 첫 번째 모델 정보 로깅
print("📝 첫 번째 모델 정보 로깅...")
mlflow.log_params({
    "model1_eta": 0.5,
    "model1_max_depth": 3,
    "model1_gamma": 4,
    "model1_min_child_weight": 6,
    "model1_subsample": 0.8,
    "model1_num_round": 50,
    "model1_training_job": xgb1.latest_training_job.name
})

# 두 번째 모델 정보 로깅
print("📝 두 번째 모델 정보 로깅...")
mlflow.log_params({
    "model2_eta": 0.1,
    "model2_max_depth": 3,
    "model2_gamma": 2,
    "model2_min_child_weight": 3,
    "model2_subsample": 0.4,
    "model2_num_round": 100,
    "model2_training_job": xgb2.latest_training_job.name
})

# 태그 설정
mlflow.set_tags({
    # 'mlflow.user': user_profile_name,
    'mlflow.source.type': 'TRAINING_JOB',
    'model.algorithm': 'XGBoost',
    'model.framework': 'Built-in',
    'training.approach': 'hyperparameter_comparison'
})

# 메트릭 로깅 시도
try:
    # 첫 번째 모델 메트릭
    metrics1 = xgb1.training_job_analytics.dataframe('auc')
    for _, row in metrics1.iterrows():
        if row['metric_name'] in ['train:auc', 'validation:auc']:
            mlflow.log_metric(f"model1_{row['metric_name'].replace(':', '_')}", row['value'])
    
    # 두 번째 모델 메트릭
    metrics2 = xgb2.training_job_analytics.dataframe('auc')
    for _, row in metrics2.iterrows():
        if row['metric_name'] in ['train:auc', 'validation:auc']:
            mlflow.log_metric(f"model2_{row['metric_name'].replace(':', '_')}", row['value'])
            
    print("✅ 메트릭 로깅 완료")
except Exception as e:
    print(f"⚠️ 메트릭 로깅 실패: {e}")

# 모델 아티팩트 로깅 시도
try:
    model1 = load_model(xgb1.model_data)
    mlflow.xgboost.log_model(model1, artifact_path="model1")
    
    model2 = load_model(xgb2.model_data)
    mlflow.xgboost.log_model(model2, artifact_path="model2")
    
    print("✅ 모델 아티팩트 로깅 완료")
except Exception as e:
    print(f"⚠️ 모델 로깅 실패: {e}")

print(f"✅ MLflow 로깅 완료 (Run ID: {run_id})")

In [ ]:
# 실행 이름 생성
timestamp = strftime('%d-%H-%M-%S', gmtime())

test_name_1 = "conservative-hyperparams"
run_name_1 = f"{test_name_1}-training-{timestamp}"

test_name_2 = "aggressive-hyperparams"
run_name_2 = f"{test_name_2}-training-{timestamp}"

print(f"✅ 실행 이름 생성 완료")
print(f"   - 첫 번째 실행: {run_name_1}")
print(f"   - 두 번째 실행: {run_name_2}")

## 7. 모델 등록

훈련된 모델들을 MLflow Model Registry에 등록하여 버전 관리를 수행합니다.

In [ ]:
# 등록할 모델 이름 설정
registered_model_name = f"bank-marketing-model-{experiment_name.split('-')[-1]}"

print(f"✅ 모델 등록 이름: {registered_model_name}")

In [ ]:
# 첫 번째 모델 등록
print("📋 첫 번째 모델을 Model Registry에 등록 중...")

model_uri_1 = f"runs:/{run_id}/model1"  # run_id 사용, model1 경로
registered_model_version_1 = mlflow.register_model(
    model_uri_1, 
    registered_model_name
)

print(f"✅ 첫 번째 모델 등록 완료")
print(f"   - 모델 이름: {registered_model_name}")
print(f"   - 버전: {registered_model_version_1.version}")

In [ ]:
# 두 번째 모델 등록
print("📋 두 번째 모델을 Model Registry에 등록 중...")

model_uri_2 = f"runs:/{run_id}/model2"  # run_id 사용, model2 경로
registered_model_version_2 = mlflow.register_model(
    model_uri_2, 
    registered_model_name
)

print(f"✅ 두 번째 모델 등록 완료")
print(f"   - 모델 이름: {registered_model_name}")
print(f"   - 버전: {registered_model_version_2.version}")

## 8. 훈련 결과 확인

MLflow UI에서 훈련 결과를 확인할 수 있는 링크를 생성합니다.

In [ ]:
# 최신 실행 정보 가져오기
experiment_id = mlflow.active_run().info.experiment_id

# 최신 실행 정보 가져오기
latest_runs = mlflow.search_runs(
    experiment_ids=[experiment_id], 
    max_results=2, 
    order_by=["attributes.start_time DESC"]
)

# MLflow UI 링크 생성
presigned_url = sess.sagemaker_client.create_presigned_mlflow_tracking_server_url(
    TrackingServerName=mlflow_name,
    ExpiresInSeconds=60,
    SessionExpirationDurationInSeconds=1800
)['AuthorizedUrl']

mlflow_experiment_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}"

print("✅ MLflow UI 링크 생성 완료")
print(f"   - 실험 페이지: {mlflow_experiment_link}")



In [ ]:
# MLflow UI 자동 열기
print("🌐 MLflow UI를 새 탭에서 열고 있습니다...")
display(Javascript(f'window.open("{mlflow_experiment_link}");'))

print("\n📊 MLflow UI에서 다음을 확인할 수 있습니다:")
print("   - 두 모델의 하이퍼파라미터 비교")
print("   - 훈련/검증 AUC 메트릭 비교")
print("   - 모델 아티팩트 및 메타데이터")
print("   - 훈련 작업 링크")

## 9. Script Mode를 사용한 커스텀 훈련

더 세밀한 제어와 자동 로깅을 위해 Script Mode를 사용하여 커스텀 훈련 스크립트로 모델을 훈련합니다.

### Script Mode의 장점
- 커스텀 훈련 로직 구현 가능
- MLflow 자동 로깅 기능 활용
- 분산 훈련 지원
- 더 유연한 하이퍼파라미터 튜닝

In [ ]:
# 훈련 스크립트 디렉토리 생성
!mkdir -p './training'

print("✅ 훈련 스크립트 디렉토리 생성 완료")

In [ ]:
%%writefile './training/requirements.txt'
mlflow==2.13.2
sagemaker-mlflow==0.1.0
sagemaker-studio

### 커스텀 훈련 스크립트 작성

MLflow 자동 로깅과 SageMaker 환경을 활용하는 훈련 스크립트를 작성합니다.

In [ ]:
%%writefile ./training/train.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl
import subprocess
import sys

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def install_requirements():
    """필수 패키지 설치"""
    try:
        logger.info("Installing MLflow packages...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'mlflow==2.13.2', '--quiet'])
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-mlflow==0.1.0', '--quiet'])
        logger.info("MLflow packages installed successfully")
    except Exception as e:
        logger.warning(f"Package installation failed: {e}")
        logger.info("Continuing without MLflow...")

# 패키지 설치
install_requirements()

try:
    from sagemaker_containers import entry_point
    from sagemaker_xgboost_container.data_utils import get_dmatrix
    from sagemaker_xgboost_container import distributed
    from sklearn.metrics import roc_auc_score
    import xgboost as xgb
    import mlflow
    MLFLOW_AVAILABLE = True
except ImportError as e:
    logger.warning(f"Import error: {e}")
    MLFLOW_AVAILABLE = False

from time import gmtime, strftime

def _xgb_train(params, dtrain, dval, evals, num_boost_round, model_dir, is_master):
    """XGBoost 훈련 함수"""
    logger.info(f"Starting XGBoost training with params: {params}")
    
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        evals=evals,
        num_boost_round=num_boost_round,
        verbose_eval=False
    )

    # 메트릭 계산
    val_auc = roc_auc_score(dval.get_label(), booster.predict(dval))
    train_auc = roc_auc_score(dtrain.get_label(), booster.predict(dtrain))
    
    logger.info(f"Training AUC: {train_auc:.4f}")
    logger.info(f"Validation AUC: {val_auc:.4f}")
    
    # MLflow 로깅 (사용 가능한 경우)
    if MLFLOW_AVAILABLE and os.getenv('MLFLOW_TRACKING_ARN'):
        try:
            mlflow.log_params(params)
            mlflow.log_metrics({"validation_auc": val_auc, "train_auc": train_auc})
        except Exception as e:
            logger.warning(f"MLflow logging failed: {e}")
    
    # SageMaker 메트릭 출력
    print(f"[0]#011train-auc:{train_auc}#011validation-auc:{val_auc}")
    
    # 모델 저장
    if is_master:
        model_location = os.path.join(model_dir, 'xgboost-model')
        with open(model_location, 'wb') as f:
            pkl.dump(booster, f)
        logger.info(f"Model saved to: {model_location}")

def parse_args():
    """명령행 인수 파싱"""
    parser = argparse.ArgumentParser()
    
    # 하이퍼파라미터
    parser.add_argument('--max_depth', type=int, default=3)
    parser.add_argument('--eta', type=float, default=0.1)
    parser.add_argument('--gamma', type=int, default=0)
    parser.add_argument('--min_child_weight', type=float, default=1)
    parser.add_argument('--subsample', type=float, default=1.0)
    parser.add_argument('--colsample_bytree', type=float, default=1.0)
    parser.add_argument('--verbosity', type=int, default=1)
    parser.add_argument('--objective', type=str, default='binary:logistic')
    parser.add_argument('--num_round', type=int, default=100)
    parser.add_argument('--early_stopping_rounds', type=int, default=10)
    parser.add_argument('--tree_method', type=str, default="auto")
    parser.add_argument('--predictor', type=str, default="auto")

    # SageMaker 환경 변수
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))
    parser.add_argument('--sm_training_env', type=str, default=os.environ.get('SM_TRAINING_ENV'))
    
    return parser.parse_known_args()

def main():
    """메인 함수"""
    try:
        args, _ = parse_args()
        logger.info("Training started")
        logger.info(f"Hyperparameters: eta={args.eta}, max_depth={args.max_depth}, num_round={args.num_round}")

        # 환경 변수 확인
        suffix = strftime('%d-%H-%M-%S', gmtime())
        user_profile_name = os.getenv('USER', 'sagemaker-user')
        experiment_name = os.getenv('MLFLOW_EXPERIMENT_NAME')
        region = os.getenv('REGION')
        mlflow_arn = os.getenv('MLFLOW_TRACKING_ARN')

        logger.info(f"Environment - User: {user_profile_name}, Region: {region}")
        
        # SageMaker 환경 정보
        sm_hosts = json.loads(args.sm_hosts) if args.sm_hosts else ['localhost']
        sm_current_host = args.sm_current_host or 'localhost'
        sm_training_env = json.loads(args.sm_training_env) if args.sm_training_env else {}
        
        # 데이터 로드
        logger.info("Loading training data...")
        dtrain = get_dmatrix(args.train, 'CSV')
        dval = get_dmatrix(args.validation, 'CSV')
        watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]
        
        logger.info(f"Data loaded - Train: {dtrain.num_row()} rows, Validation: {dval.num_row()} rows")
        
        # MLflow 설정 (사용 가능한 경우)
        if MLFLOW_AVAILABLE and mlflow_arn:
            try:
                mlflow.set_tracking_uri(mlflow_arn)
                mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"script-training-{suffix}")
                mlflow.xgboost.autolog(log_model_signatures=False, log_datasets=False)
                logger.info("MLflow configured successfully")
            except Exception as e:
                logger.warning(f"MLflow setup failed: {e}")

        # 훈련 하이퍼파라미터 설정
        train_hp = {
            'max_depth': args.max_depth,
            'eta': args.eta,
            'gamma': args.gamma,
            'min_child_weight': args.min_child_weight,
            'subsample': args.subsample,
            'colsample_bytree': args.colsample_bytree,
            'verbosity': 0,  # 로그 출력 최소화
            'objective': args.objective,
            'tree_method': args.tree_method,
            'predictor': args.predictor,
        }

        xgb_train_args = {
            'params': train_hp,
            'dtrain': dtrain,
            'dval': dval,
            'evals': watchlist,
            'num_boost_round': args.num_round,
            'model_dir': args.model_dir
        }

        # MLflow 실행 시작 (사용 가능한 경우)
        if MLFLOW_AVAILABLE and mlflow_arn:
            try:
                with mlflow.start_run(
                    run_name=f"script-mode-training-{suffix}",
                    description="SageMaker Script Mode XGBoost Training"
                ) as run:
                    # 태그 설정
                    mlflow.set_tags({
                        'mlflow.user': user_profile_name,
                        'mlflow.source.type': 'TRAINING_JOB',
                        'model.framework': 'Script-Mode',
                        'training.mode': 'script'
                    })
                    
                    # 훈련 실행
                    _execute_training(sm_hosts, sm_current_host, dtrain, xgb_train_args)
            except Exception as e:
                logger.warning(f"MLflow run failed: {e}")
                # MLflow 없이 훈련 진행
                _execute_training(sm_hosts, sm_current_host, dtrain, xgb_train_args)
        else:
            logger.info("Training without MLflow")
            _execute_training(sm_hosts, sm_current_host, dtrain, xgb_train_args)
        
        logger.info("Training completed successfully")
        
    except Exception as e:
        logger.error(f"Training failed: {str(e)}")
        raise e

def _execute_training(sm_hosts, sm_current_host, dtrain, xgb_train_args):
    """훈련 실행 함수"""
    if len(sm_hosts) > 1:
        logger.info(f"Distributed training with {len(sm_hosts)} hosts")
        entry_point._wait_hostname_resolution()
        distributed.rabit_run(
            exec_fun=_xgb_train,
            args=xgb_train_args,
            include_in_training=(dtrain is not None),
            hosts=sm_hosts,
            current_host=sm_current_host,
            update_rabit_args=True
        )
    else:
        logger.info("Single node training")
        if dtrain:
            xgb_train_args['is_master'] = True
            _xgb_train(**xgb_train_args)
        else:
            raise ValueError("No training data available")

def model_fn(model_dir):
    """모델 로드 함수 (추론용)"""
    model_file = 'xgboost-model'
    model_path = os.path.join(model_dir, model_file)
    with open(model_path, 'rb') as f:
        booster = pkl.load(f)
    return booster

if __name__ == '__main__':
    main()

### 하이퍼파라미터 및 환경 변수 설정

Script Mode 훈련을 위한 하이퍼파라미터와 환경 변수를 설정합니다.

In [ ]:
# Script Mode용 하이퍼파라미터 설정
script_hyperparams = {
    'num_round': 75,
    'max_depth': 4,
    'eta': 0.2,
    'gamma': 1,
    'objective': 'binary:logistic',
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 2,
    'early_stopping_rounds': 10,
    'verbosity': 1
}

# 환경 변수 설정 (Project 클래스 활용)
env_variables = {
    'MLFLOW_TRACKING_ARN': mlflow_arn,  # Project에서 가져온 MLflow ARN
    'MLFLOW_EXPERIMENT_NAME': experiment_name,
    'USER': user_profile_name,
    'REGION': region,
}

print("✅ Script Mode 하이퍼파라미터 설정 완료")
print(f"   - 학습률(eta): {script_hyperparams['eta']}")
print(f"   - 최대 깊이: {script_hyperparams['max_depth']}")
print(f"   - 라운드 수: {script_hyperparams['num_round']}")
print(f"   - MLflow ARN: {env_variables['MLFLOW_TRACKING_ARN']}")

### Script Mode Estimator 생성

XGBoost Script Mode Estimator를 생성하고 훈련을 실행합니다.

In [ ]:
# Script Mode Estimator 생성 (Unified Studio 환경 최적화)
xgb_script_mode = XGBoost(
    entry_point='train.py',  # 새로운 스크립트 사용
    source_dir='./training',
    framework_version="1.7-1",  
    hyperparameters=script_hyperparams,
    role=role,  # Project에서 가져온 역할
    instance_count=1, 
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/script-output/',
    code_location=f's3://{bucket}/{prefix}/code/',  # 코드 업로드 경로 명시
    base_job_name="unified-studio-xgb-training",
    environment=env_variables
)

print("✅ Unified Studio Script Mode Estimator 생성 완료")
print(f"   - 프레임워크 버전: 1.7-1")
print(f"   - 인스턴스 타입: ml.m5.large")
print(f"   - 출력 경로: s3://{bucket}/{prefix}/script-output/")
print(f"   - 코드 업로드 경로: s3://{bucket}/{prefix}/code/")
print(f"   - Unified Studio 최적화 스크립트 사용")

### Script Mode 훈련 실행

커스텀 스크립트를 사용하여 모델을 훈련합니다.

In [ ]:
# Script Mode 훈련 실행
print("🚀 Script Mode 훈련 시작...")
print("   - 커스텀 훈련 스크립트 사용")
print("   - MLflow 자동 로깅 활성화")
print("   - 실시간 메트릭 추적")

# 로그 활성화로 디버깅
xgb_script_mode.fit(training_inputs, wait=True, logs=True)

print(f"✅ Script Mode 훈련 완료!")
print(f"   - 훈련 작업: {xgb_script_mode.latest_training_job.name}")
print(f"   - 모델 S3 URI: {xgb_script_mode.model_data}")

### Script Mode 결과 확인

Script Mode로 훈련된 모델의 결과를 MLflow UI에서 확인합니다.

In [ ]:
# Script Mode 훈련 결과 확인
try:
    # 최신 실행 정보 가져오기
    latest_runs = mlflow.search_runs(
        experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id], 
        max_results=1, 
        order_by=["attributes.start_time DESC"]
    )
    
    if not latest_runs.empty:
        latest_run_id = latest_runs['run_id'][0]
        
        # MLflow UI 링크 생성
        presigned_url = sess.sagemaker_client.create_presigned_mlflow_tracking_server_url(
            TrackingServerName=mlflow_name,
            ExpiresInSeconds=60,
            SessionExpirationDurationInSeconds=1800
        )['AuthorizedUrl']
        
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
        script_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}/runs/{latest_run_id}"
        
        print("✅ Script Mode 훈련 결과 확인")
        print(f"   - 최신 실행 ID: {latest_run_id}")
        print(f"   - MLflow UI: {script_run_link}")
        
        # MLflow UI 자동 열기
        print("\n🌐 MLflow UI를 새 탭에서 열고 있습니다...")
        display(Javascript(f'window.open("{script_run_link}");'))
        
    else:
        print("⚠️ MLflow 실행 정보를 찾을 수 없습니다.")
        
except Exception as e:
    print(f"⚠️ MLflow 결과 확인 중 오류: {e}")

print("\n📊 Script Mode에서 확인 가능한 정보:")
print("   - 자동 로깅된 하이퍼파라미터")
print("   - 훈련/검증 AUC 메트릭")
print("   - 모델 아티팩트 (자동 저장)")
print("   - 훈련 과정 로그")
print("   - SageMaker 훈련 작업 링크")

In [ ]:
# 다음 노트북에서 사용할 변수들 저장
%store run_name_1
%store registered_model_version_1
%store registered_model_version_2
%store registered_model_name

print("✅ 변수 저장 완료")
print("\n📋 저장된 정보:")
print(f"   - 등록된 모델 이름: {registered_model_name}")
print(f"   - 모델 버전 1: {registered_model_version_1.version}")
print(f"   - 모델 버전 2: {registered_model_version_2.version}")
print(f"   - 실행 ID 1: {run_id}")

## ✅ 모델 훈련 완료

SageMaker Training Job을 사용한 XGBoost 모델 훈련이 성공적으로 완료되었습니다!

### 완료된 작업
- ✅ 두 가지 하이퍼파라미터 설정으로 XGBoost 모델 훈련
- ✅ MLflow를 통한 실험 추적 및 메트릭 로깅
- ✅ 훈련된 모델을 Model Registry에 등록
- ✅ 모델 성능 비교를 위한 MLflow UI 링크 생성

### 훈련된 모델
1. **보수적 모델**: 빠른 학습률(0.5), 적은 라운드(50)
2. **적극적 모델**: 느린 학습률(0.1), 많은 라운드(100)

### 다음 단계
이제 `3-test-and-deploy.ipynb` 노트북으로 진행하여:
- 훈련된 모델들의 성능 평가
- 최적 모델 선택
- SageMaker 엔드포인트로 모델 배포
- 실시간 추론 테스트

In [ ]:
# 훈련 완료 요약
print("🎉 모델 훈련 완료!")
print("=" * 50)

print(f"📊 첫 번째 모델 (보수적 하이퍼파라미터):")
print(f"   - 훈련 작업명: {xgb1.latest_training_job.name}")
print(f"   - 모델 S3 URI: {xgb1.model_data}")
print(f"   - 하이퍼파라미터: {xgb1.hyperparameters()}")

print(f"\n📊 두 번째 모델 (적극적 하이퍼파라미터):")
print(f"   - 훈련 작업명: {xgb2.latest_training_job.name}")
print(f"   - 모델 S3 URI: {xgb2.model_data}")
print(f"   - 하이퍼파라미터: {xgb2.hyperparameters()}")

print(f"\n📜 Script Mode 모델:")
print(f"   - 훈련 작업명: {xgb_script_mode.latest_training_job.name}")
print(f"   - 모델 S3 URI: {xgb_script_mode.model_data}")
print(f"   - MLflow 실행 ID: {run_id}")

print("\n✅ 모든 모델이 성공적으로 훈련되었습니다!")
print("   다음 단계: 3-model-evaluation.ipynb에서 성능 평가")

In [ ]:
# 다음 노트북에서 사용할 변수들 저장
%store run_id
#%store xgb1
#%store xgb2
#%store xgb_script_mode
%store registered_model_name
%store registered_model_version_1
%store registered_model_version_2

# 모델 URI도 별도로 저장 (배포 단계에서 사용)
model1_uri = xgb1.model_data
model2_uri = xgb2.model_data
%store model1_uri
%store model2_uri

print("✅ 변수 저장 완료")
print("\n📋 저장된 정보:")
print(f"   - MLflow 실행 ID: {run_id}")
print(f"   - 등록된 모델 이름: {registered_model_name}")
print(f"   - 모델 버전 1: {registered_model_version_1.version}")
print(f"   - 모델 버전 2: {registered_model_version_2.version}")
print(f"\n🔧 Built-in 알고리즘 모델:")
print(f"   - 첫 번째 모델 훈련 작업: {xgb1.latest_training_job.name}")
print(f"   - 두 번째 모델 훈련 작업: {xgb2.latest_training_job.name}")
print(f"   - 첫 번째 모델 S3 URI: {xgb1.model_data}")
print(f"   - 두 번째 모델 S3 URI: {xgb2.model_data}")
print(f"\n📜 Script Mode 모델:")
print(f"   - Script Mode 훈련 작업: {xgb_script_mode.latest_training_job.name}")
print(f"   - Script Mode 모델 S3 URI: {xgb_script_mode.model_data}")